In [ ]:
!pip install transformers==4.6.0
!pip install datasets
!pip install torch torchvision torchaudio
!pip install keras
!pip install tensorflow

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1,2,3"

In [1]:
import pandas as pd
import numpy as np

In [23]:
forum_test = pd.read_csv("1-forum-only/Forum_label_test.csv", index_col="Unnamed: 0")
synthetic_test = pd.read_csv("2-synthetic-only/synthetic_full_output_test.csv", index_col="Unnamed: 0")

forum_train = pd.read_csv("1-forum-only/Forum_label_train.csv", index_col="Unnamed: 0")
synthetic_train = pd.read_csv("2-synthetic-only/synthetic_full_output_train.csv", index_col="Unnamed: 0")

In [24]:
forum_8 = pd.read_csv("1-forum-only/forum_8_percent.csv", index_col="Unnamed: 0")
forum_8["labels"].value_counts()

0    178
1     93
2     31
Name: labels, dtype: int64

In [20]:
len(forum_8)

302

In [27]:
mixed_as_q_test = pd.concat([forum_test, synthetic_test])
mixed_as_q_test = mixed_as_q_test.sample(frac=1)
mixed_as_q_test.head()

,Sentence,labels
4719,Ja was hat eigentlich gut funktioniert ich h...,1
7883,Mein Freund hat mich beim Sport begleitet und...,1
5259,Ich werde mir das Buch „Das Abnehmen für Dumm...,2
2470,"Ich kann mein Gewicht halten, indem ich auf d...",0
12326,Ich möchte nicht mehr so viel essen und mehr ...,0


In [28]:
mixed_as_q_test.to_csv("4-mixed_lab_q/mixed_full_output_test.csv")

In [9]:
mixed_as_q_train = pd.concat([forum_train, synthetic_train])
mixed_as_q_train = mixed_as_q_train.sample(frac=1)
mixed_as_q_train.head()

,Sentence,labels
19868,Ich habe keine Ahnung wie ich das ändern kann.,0
13623,Ich bin hier um Dich auf deinem Weg zu einem ...,2
6023,"Nur wenn Du gesund bist, kannst Du auch glück...",0
19114,Ich werde mein Essen auf den Tisch stellen un...,2
20201,"Ich weiß, wie man abnimmt.",0


In [20]:
mixed_as_q_test["labels"].value_counts()

0    4167
1    1127
2     833
Name: labels, dtype: int64

In [21]:
synthetic_test["labels"].value_counts()

0    3555
1     885
2     742
Name: labels, dtype: int64

In [22]:
forum_test["labels"].value_counts()

0    612
1    242
2     91
Name: labels, dtype: int64

In [2]:
mixed_as_q_train.to_csv("4-mixed_lab_q/mixed_train_as_q.csv")

NameError: name 'mixed_as_q_train' is not defined

In [24]:
prep_for_class = pd.read_csv("orig-gpt_output_labels_as_qs.csv", index_col="Unnamed: 0")
prep_for_class.drop(columns="labels", inplace=True)

In [26]:
prep_for_class.to_csv("gpt_output_for_pred.csv")

In [12]:
predicted_train = pd.read_csv("gpt_predictions/synthetic_test_predicted_confidence.csv", index_col="Unnamed: 0")
#predicted_test = pd.read_csv("gpt_predictions/synthetic_test_predicted_confidence.csv", index_col="Unnamed: 0")

In [13]:
predicted_train.rename(columns={"sentences":"Sentence"}, inplace=True)
predicted_train.head()

,Sentence,0_R,1_TS,2_C
0,Ich wünsche mir mehr Kontrolle über meine Ern...,0.999254,0.000456,0.000290
1,Ich habe mich ein wenig gesünder ernährt.,0.000600,0.999124,0.000276
2,"Welche Möglichkeiten siehst du, um etwas gege...",0.999356,0.000381,0.000263
3,Aber das hat nicht geklappt.,0.136079,0.862327,0.001594
4,Ja.,0.997365,0.001643,0.000993


In [14]:
R = predicted_train[predicted_train["0_R"] >= 0.80].copy()
R["labels"] = 0
TS = predicted_train[predicted_train["1_TS"] >= 0.80].copy()
TS["labels"] = 1
C = predicted_train[predicted_train["2_C"] >= 0.80].copy()
C["labels"] = 2

In [15]:
synthetic_filtered = pd.concat([R, TS, C, forum_test])
synthetic_filtered.drop(columns=["0_R", "1_TS", "2_C"], inplace=True)
synthetic_filtered = synthetic_filtered.sample(frac=1)

In [16]:
synthetic_filtered.tail()

,Sentence,labels
2584,"Dann werde ich mehr Geld haben, um zu teurem ...",2
2595,Ich werde nicht mehr zu viel Zucker essen.,2
682,"Ich glaube, dass ich abnehmen muss.",0
590,Ich habe meine Ernährung umgestellt.,1
29,"Mit Therapie nehme ich auch nicht ab, das hab...",0


In [17]:
synthetic_filtered["labels"].value_counts()

0    4002
1    1079
2     768
Name: labels, dtype: int64

In [18]:
synthetic_filtered.to_csv("3-mixed_predicted/confidence_80_test.csv")

In [6]:
percent_forum = pd.read_csv("1-forum-only/forum_10_percent.csv", index_col="Unnamed: 0")
synthetic_train = pd.read_csv("2-synthetic-only/synthetic_train_gpt_labels_as_qs.csv", index_col="Unnamed: 0")
percent_mix = pd.concat([percent_forum, synthetic_train]).sample(frac=1)
percent_mix.to_csv("4-mixed_lab_q/10_percent_mixed_q.csv")

In [11]:
percent_mix.to_csv("3-mixed_predicted/4_percent_mixed.csv")

In [27]:
R_test = predicted_test[predicted_test["0_R"] > 0.95].copy()
R_test["labels"] = 0
TS_test = predicted_test[predicted_test["1_TS"] > 0.95].copy()
TS_test["labels"] = 1
C_test = predicted_test[predicted_test["2_C"] > 0.95].copy()
C_test["labels"] = 2

synthetic_filtered_test = pd.concat([R_test, TS_test, C_test])
synthetic_filtered_test.drop(columns=["0_R", "1_TS", "2_C"], inplace=True)
synthetic_filtered_test = synthetic_filtered_test.sample(frac=1)

In [35]:
synthetic_filtered.rename(columns={"sentences":"Sentence"}, inplace=True)
synthetic_filtered_test.rename(columns={"sentences":"Sentence"}, inplace=True)

In [36]:
synthetic_filtered.to_csv("synthetic_train_labels_pred.csv")
synthetic_filtered_test.to_csv("synthetic_test_labels_pred.csv")

In [39]:
filtered_mixed_train = pd.concat([synthetic_filtered, forum_train])
filtered_mixed_train = filtered_mixed_train.sample(frac=1)
filtered_mixed_test = pd.concat([synthetic_filtered_test, forum_test])
filtered_mixed_test = filtered_mixed_test.sample(frac=1)

In [46]:
filtered_mixed_train.to_csv("3-mixed_predicted/mixed_train_as_pred.csv")
filtered_mixed_test.to_csv("3-mixed_predicted/mixed_test_as_pred.csv")